# Posner analyze

## Import module

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import glob
import math
import scipy.signal as signal

## Get posner stage

In [ ]:
def Stage(df):
    '''
    return: dataframe denoted with each stage

    inputs:
        df: dataframe
    '''
    OwO = pd.read_csv(df, index_col='Time_Host')
                
    OwO = OwO.drop(columns=['CH1_Oxy', 'CH1_Deoxy', 'CH4_Oxy', 'CH4_Deoxy'])
    OwO['CH2_HbT'] = OwO['CH2_Oxy'] + OwO['CH2_Deoxy']
    OwO['CH3_HbT'] = OwO['CH3_Oxy'] + OwO['CH3_Deoxy']

    # set stage
    posner_stage_time = [0, 60, OwO[OwO['posner_trail'] == 0].index.max(), OwO[OwO['posner_trail'] == 0].index.max()+60, OwO[OwO['posner_trail'] == 1].index.max(), OwO[OwO['posner_trail'] == 1].index.max()+60, OwO[OwO['posner_trail'] == 2].index.max(), OwO[OwO['posner_trail'] == 2].index.max()+200]
    OwO['posner_stage'] = 0
    for i in range(0, 7):
        OwO.posner_stage.loc[posner_stage_time[i]:posner_stage_time[i+1]] = i

## Get activation

In [ ]:
def Feature(df):
    '''
    return: the activarion of each stage and difference of peak of between two stages

    inputs:
        df: dataframe
    '''
    stage_peak = df.drop(columns=['posner_trail', 'posner_task']).groupby('posner_stage').agg(max).stack()
    stage_activation = stage_peak.sub(df.drop(columns=['posner_trail', 'posner_task']).groupby('posner_stage').agg(min).stack())
    stage_activation.index = stage_activation.index.map(lambda col: 'stage' + str(col[0]) + '_' + str(col[1]) + '_activation')

    features = stage_activation.to_frame().T

    return features

# Main

In [ ]:
group_dir = []
for dirPath, dirNames, fileNames in os.walk(r'path\2_processed_data'):
    group_dir.append(dirPath)
all_activation = pd.DataFrame()
all_pre_activation = pd.DataFrame()
all_post_activation = pd.DataFrame()
case_num=0

for group in group_dir:
    split_list = group.split('\\')
    data_link = os.path.join('..\\', split_list[-4], split_list[-3], split_list[-2], split_list[-1], "*.csv")
    file_list = glob.glob(data_link)
    if len(file_list)>0:
        for file in file_list:
            if 'post' in file:
                OwO = Stage(file)
                post_features = Feature(OwO)

            elif 'pre' in file:
                OwO = Stage(file)
                pre_features = Feature(OwO)
            
        post_features['group'] = split_list[-2]
        pre_features['group'] = split_list[-2]
        post_features['case'] = case_num
        pre_features['case'] = case_num
        case_num+=1

        all_pre_activation = pd.concat([all_pre_activation, pre_features])
        all_post_activation = pd.concat([all_post_activation, post_features])
    else:
        pass

all_pre_activation.to_csv(r'path\pre_activation.csv', index=False)
all_post_activation.to_csv(r'path\post_activation.csv', index=False)